# Credit card fraud detector

## Investigate and process the data

Let's start by downloading and reading in the credit card fraud data set.

In [1]:
%%bash
wget https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip
unzip creditcardfraud.zip

Archive:  creditcardfraud.zip


--2019-12-01 17:19:34--  https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.192.40
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.192.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip.1’

     0K .......... .......... .......... .......... ..........  0%  349K 3m14s
    50K .......... .......... .......... .......... ..........  0% 78.0M 97s
   100K .......... .......... .......... .......... ..........  0%  700K 97s
   150K .......... .......... .......... .......... ..........  0% 84.2M 73s
   200K .......... .......... .......... .......... ..........  0%  706K 77s
   250K .......... .......... .......... .......... ..........  0%  127M 64s
   300K .......... .......... .......... .......... ..........  0%  122M 55s
   350K .......... ..........

In [2]:
import numpy as np 
import pandas as pd

data = pd.read_csv('creditcard.csv', delimiter=',')

Let's take a peek at our data (we only show a subset of the columns in the table):

In [3]:
print(data.columns)
data[['Time', 'V1', 'V2', 'V27', 'V28', 'Amount', 'Class']].describe()

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


,Time,V1,V2,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,3.161220e+01,3.384781e+01,25691.160000,1.000000


The class column corresponds to whether or not a transaction is fradulent. We see that the majority of data is non-fraudulant with only $492$ ($.173\%$) of the data corresponding to fraudulant examples.

In [4]:
nonfrauds, frauds = data.groupby('Class').size()
print('Number of frauds: ', frauds)
print('Number of non-frauds: ', nonfrauds)
print('Percentage of fradulent data:', 100.*frauds/(frauds + nonfrauds))

Number of frauds:  492
Number of non-frauds:  284315
Percentage of fradulent data: 0.1727485630620034


This dataset has 28 columns, $V_i$ for $i=1..28$ of anonymized features along with columns for time, amount, and class. We already know that the columns $V_i$ have been normalized to have $0$ mean and unit standard deviation as the result of a PCA. You can read more about PCA here:. 

Tip: For our dataset this amount of preprocessing will give us reasonable accuracy, but it's important to note that there are more preprocessing steps one can use to improve accuracy . For unbalanced data sets like ours where the positive (fraudulent) examples occur much less frequently than the negative (legitimate) examples, we may try “over-sampling” the minority dataset by generating synthetic data (read about SMOTE in Data Mining for Imbalanced Datasets: An Overview (https://link.springer.com/chapter/10.1007%2F0-387-25465-X_40) or undersampling the majority class by using ensemble methods (see http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.68.6858&rep=rep1&type=pdfor).

In [5]:
feature_columns = data.columns[:-1]
label_column = data.columns[-1]

features = data[feature_columns].values.astype('float32')
labels = (data[label_column].values).astype('float32')

Let's do some analysis and discuss different ways we can preprocess our data. Let's discuss the way in which this data was preprocessed.

## SageMaker Linear Learner

### Prepare Data and Upload to S3

The Amazon common libraries provide utilities to convert NumPy n-dimensional arrays into a the Record-IO format which SageMaker uses for a concise representation of features and labels. The Record-IO format is implemented via protocol buffer so the serialization is very efficient.

In [6]:
import io
import sagemaker.amazon.common as smac

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, features, labels)
buf.seek(0);

Now we upload the data to S3 using boto3.

In [7]:
import boto3
import os
import sagemaker

session = sagemaker.Session()

bucket = session.default_bucket()
#sagemaker_iam_role = get_execution_role()

prefix = 'linear-learner'
key = 'recordio-pb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)

s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('Uploaded training data location: {}'.format(s3_train_data))

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

Uploaded training data location: s3://sagemaker-us-east-1-118598624177/linear-learner/train/recordio-pb-data
Training artifacts will be uploaded to: s3://sagemaker-us-east-1-118598624177/linear-learner/output


Now we train a Linear Learner using SageMaker's built-in algorithm. To specify the Linear Learner algorithm, we use a utility function to obtain it's URI. A complete list of build-in algorithms is found here: https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html

In [8]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, 'linear-learner')

SageMaker abstracts training with Estimators. We can pass container, and all parameters to the estimator, as well as the hyperparameters for the linear learner and fit the estimator to the data in S3.

In [9]:
from sagemaker import get_execution_role

linear = sagemaker.estimator.Estimator(container,
                                       role = get_execution_role(), 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c5.4xlarge',
                                       output_path=output_location,
                                       sagemaker_session=session)
linear.set_hyperparameters(feature_dim=features.shape[1],
                           predictor_type='binary_classifier',
                           mini_batch_size=200)

linear.fit({'train': s3_train_data})

2019-12-01 17:19:50 Starting - Starting the training job...
2019-12-01 17:19:51 Starting - Launching requested ML instances......
2019-12-01 17:20:54 Starting - Preparing the instances for training...
2019-12-01 17:21:48 Downloading - Downloading input data
2019-12-01 17:21:48 Training - Downloading the training image..Docker entrypoint called with argument(s): train
[12/01/2019 17:22:03 INFO 140346379577152] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'auto'

[2019-12-01 17:22:44.165] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 20444, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.007420677197560815, "sum": 0.007420677197560815, "min": 0.007420677197560815}}, "EndTime": 1575220964.16514, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1575220964.165084}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0057146841035227115, "sum": 0.0057146841035227115, "min": 0.0057146841035227115}}, "EndTime": 1575220964.165192, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1575220964.165183}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.007641509

[2019-12-01 17:23:04.647] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 20477, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006549437720638574, "sum": 0.006549437720638574, "min": 0.006549437720638574}}, "EndTime": 1575220984.647735, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1575220984.647665}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005025194320733544, "sum": 0.005025194320733544, "min": 0.005025194320733544}}, "EndTime": 1575220984.6478, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1575220984.64779}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00668306914777

[2019-12-01 17:23:25.336] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 20684, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006084247275852085, "sum": 0.006084247275852085, "min": 0.006084247275852085}}, "EndTime": 1575221005.33705, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1575221005.336995}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0047343518962858645, "sum": 0.0047343518962858645, "min": 0.0047343518962858645}}, "EndTime": 1575221005.337104, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1575221005.337096}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006080260

[2019-12-01 17:23:45.551] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 20209, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005935613360371064, "sum": 0.005935613360371064, "min": 0.005935613360371064}}, "EndTime": 1575221025.551572, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1575221025.551508}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00456818926897622, "sum": 0.00456818926897622, "min": 0.00456818926897622}}, "EndTime": 1575221025.551631, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1575221025.551621}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0058136687935

...

[2019-12-01 17:26:07.790] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 20305, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005682079380232935, "sum": 0.005682079380232935, "min": 0.005682079380232935}}, "EndTime": 1575221167.790547, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1575221167.790491}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004223928141274902, "sum": 0.004223928141274902, "min": 0.004223928141274902}}, "EndTime": 1575221167.790604, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1575221167.790595}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00461248

[2019-12-01 17:26:27.960] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 20165, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0056506357096747775, "sum": 0.0056506357096747775, "min": 0.0056506357096747775}}, "EndTime": 1575221187.960787, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1575221187.960735}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004216853662955135, "sum": 0.004216853662955135, "min": 0.004216853662955135}}, "EndTime": 1575221187.960841, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1575221187.96083}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004541

[2019-12-01 17:26:48.081] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 20116, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005620591817436782, "sum": 0.005620591817436782, "min": 0.005620591817436782}}, "EndTime": 1575221208.081502, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1575221208.081446}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004178302496082406, "sum": 0.004178302496082406, "min": 0.004178302496082406}}, "EndTime": 1575221208.081558, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1575221208.081548}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00448250

[2019-12-01 17:27:08.573] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 20486, "num_examples": 1425, "num_bytes": 47847576}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005591145332803259, "sum": 0.005591145332803259, "min": 0.005591145332803259}}, "EndTime": 1575221228.573138, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1575221228.573074}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004147620221084569, "sum": 0.004147620221084569, "min": 0.004147620221084569}}, "EndTime": 1575221228.573194, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1575221228.573183}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00443457


2019-12-01 17:27:20 Uploading - Uploading generated training model
2019-12-01 17:27:20 Completed - Training job completed
Training seconds: 343
Billable seconds: 343


### Host Linear Classifier

Now we deploy the estimator to and endpoint.

In [27]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor = linear.deploy(initial_instance_count=1,
                                 endpoint_name="fraud-detection-endpoint-LL",
                                 instance_type='ml.c5.xlarge') #'ml.c5.18xlarge'
# Specify input and output formats.
linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

Using already existing model: linear-learner-2019-12-01-17-19-50-203


---------------------------------------------------------------------------------------!

## Clean up

We will leave the prediction endpoint running at the end of this notebook so we can handle incoming event streams. However, don't forget to delete the prediction endpoint when you're done. You can do that at the Amazon SageMaker console in the Endpoints page. Or you can run `linear_predictor.delete_endpoint()`


## Data Acknowledgements

The dataset used to demonstrated the fraud detection solution has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on https://www.researchgate.net/project/Fraud-detection-5 and the page of the [DefeatFraud](https://mlg.ulb.ac.be/wordpress/portfolio_page/defeatfraud-assessment-and-validation-of-deep-feature-engineering-and-learning-solutions-for-fraud-detection/) project
We cite the following works:
* Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015
* Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon
* Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE
* Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)
* Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier
* Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing

In [28]:
import time

In [29]:
import json
import os
import boto3
import random
import datetime
import re
import time

def lambda_handler(event, context):
    data_payload = get_data(event, context)
    if not data_payload:
        return
    tic = time.time()
    pred = get_fraud_prediction(data_payload)
    toc = time.time()
    print("prediction + postprocessing latency", toc-tic)
    transformed_data = postprocess_event(event, pred)
    response = store_data_prediction(transformed_data)
    print(response)

def get_data(event, context):
    if random.random() < 0.15:
        return
    non_fraud_example = [1.00000000e+00, -9.66271698e-01, -1.85226008e-01, 1.79299331e+00, -8.63291264e-01, -1.03088794e-02, 1.24720311e+00, 2.37608939e-01,
                         3.77435863e-01, -1.38702404e+00, -5.49519211e-02, -2.26487264e-01, 1.78228229e-01, 5.07756889e-01, -2.87923753e-01, -6.31418109e-01,
                         -1.05964720e+00, -6.84092760e-01, 1.96577501e+00, -1.23262203e+00, -2.08037779e-01, -1.08300455e-01, 5.27359685e-03, -1.90320522e-01,
                         -1.17557538e+00, 6.47376060e-01, -2.21928850e-01, 6.27228469e-02, 6.14576302e-02, 1.23500000e+02]
    fraud_example = [4.0600000e+02, -2.3122265e+00, 1.9519920e+00, -1.6098508e+00, 3.9979055e+00, -5.2218789e-01, -1.4265453e+00, -2.5373874e+00,
                     1.3916572e+00, -2.7700894e+00, -2.7722721e+00, 3.2020333e+00, -2.8999074e+00, -5.9522188e-01, -4.2892537e+00, 3.8972411e-01, -1.1407472e+00,
                     -2.8300557e+00, -1.6822468e-02, 4.1695571e-01, 1.2691055e-01, 5.1723236e-01, -3.5049368e-02, -4.6521106e-01, 3.2019821e-01, 4.4519167e-02,
                     1.7783980e-01, 2.6114500e-01, -1.4327587e-01, 0.0000000e+00]
    examples = [fraud_example, non_fraud_example]
    idx = 1
    if random.random() < 0.05:
        idx = 0
    return ','.join(map(str, examples[idx]))

def get_fraud_prediction(data):
    sagemaker_endpoint_name = 'fraud-detection-endpoint-LL'
    sagemaker_runtime = boto3.client('sagemaker-runtime')
    tic = time.time()
    response = sagemaker_runtime.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='text/csv',
                                                 Body=data)
    toc = time.time()
    print("invocation latency", toc-tic)
    print(response)
    result = json.loads(response['Body'].read().decode())
    print(result)
    pred = int(result['predictions'][0]['predicted_label'])
    return predbajk

def postprocess_event(event, pred):
    millisecond_regex = r'\.\d+'
    timestamp = re.sub(millisecond_regex, '', str(datetime.datetime.now()))
    source = random.choice(['Mobile', 'Web', 'Store'])
    return [timestamp, 'random_id', source, str(pred)]

def store_data_prediction(data):
    firehose_delivery_stream = 'fraud-detection-firehose-stream'
    firehose = boto3.client('firehose', region_name=os.environ['AWS_REGION'])
    record = ','.join(data) + '\n'
    response = firehose.put_record(DeliveryStreamName=firehose_delivery_stream, Record={'Data': record})
    return response

In [30]:

sagemaker_endpoint_name = 'fraud-detection-endpoint-LL' 
sagemaker_runtime = boto3.client('sagemaker-runtime')
tic = time.time()
response = sagemaker_runtime.invoke_endpoint(EndpointName=sagemaker_endpoint_name, 
                    ContentType='text/csv', Body=data)
toc = time.time()
print("invocation latency", toc-tic)
print(response)
result = json.loads(response['Body'].read().decode())
print(result)
pred = int(result['predictions'][0]['predicted_label'])
#return pred


invocation latency 0.14750146865844727
{'ResponseMetadata': {'RequestId': '571d3511-815a-42a0-9e55-10ddde89de20', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '571d3511-815a-42a0-9e55-10ddde89de20', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Sun, 1 Dec 2019 17:53:14 GMT', 'content-type': 'application/json', 'content-length': '76'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7fee7543e780>}
{'predictions': [{'score': 0.00025917874881997705, 'predicted_label': 0.0}]}


In [31]:
sagemaker_endpoint_name = 'fraud-detection-endpoint-LL'
sagemaker_runtime = boto3.client('sagemaker-runtime')

In [32]:
tic = time.time()
N = 500
skipped = 0
warmup_flag = 0
for i in range(N):
    data = get_data(None, None)
    if data == None:
        skipped += 1
        continue
    #print(data)    
    response = sagemaker_runtime.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='text/csv', Body=data)
    warmup_flag += 1
    if warmup_flag == 1:
        print("1st invocation latency", time.time() - tic)
        tic = time.time()
toc = time.time()
N = N - skipped
print("invocation latency", (toc-tic)/N)
print(response)
result = json.loads(response['Body'].read().decode())
print(result)
pred = int(result['predictions'][0]['predicted_label'])

1st invocation latency 0.0887453556060791
invocation latency 0.0254063123748416
{'ResponseMetadata': {'RequestId': '4bf66369-4af6-4ef7-b7fe-521952c2fafc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4bf66369-4af6-4ef7-b7fe-521952c2fafc', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Sun, 1 Dec 2019 17:53:25 GMT', 'content-type': 'application/json', 'content-length': '76'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7fee75360320>}
{'predictions': [{'score': 0.00025917874881997705, 'predicted_label': 0.0}]}


In [33]:

tic = time.time()
skipped = 0
warmup_flag = 0
for i in range(N):
    data = get_data(None, None)
    if data == None:
        skipped += 1
        continue
    warmup_flag += 1
    if warmup_flag == 1:
        print("1st invocation latency", time.time() - tic)
        tic = time.time()
toc = time.time()
N = N - skipped
print("{:10.10f}".format((toc - tic)/N))
#print "{:10.4f}".format(x)


1st invocation latency 0.00010061264038085938
0.0000127588
